In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
pos_path = "/content/test_Arabic_tweets_positive_20190413 (1).tsv"
neg_path = "/content/test_Arabic_tweets_negative_20190413.tsv"

positive = pd.read_csv(pos_path, sep="\t", names=["text"])
negative = pd.read_csv(neg_path, sep="\t", names=["text"])

positive["label"] = 1
negative["label"] = 0

df = pd.concat([positive, negative], ignore_index=True)

df.head()


,text,label
0,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,1
1,صباحك خيرات ومسرات 🌸,1
2,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,1
3,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,1
4,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,1


In [ ]:
arabic_stopwords = set("""
فى في كل على عن من إلى الى إلي ايضا و أو ان أن إن ما لا نعم ثم به بها لها له لديهم لدي لديكم انت انا هو هي هم هن نحن كما كان تكون تكونوا هذه هذا هناك حيث قد لم لن لكن بحيث أجل دون عبر حول جدا حتى بين اكثر بعد قبل حين أجل عندما إذا اذا فقط بالنسبة خلال ضد منذ تحت فوق لدى أحد إحدى شيء بعض كثير جميع أي اي كيف غير تلك ذلك وهكذا معها معه معها أنها انه إنهم إننا أنكم الذين التي اللواتي اللائي الذين لك لنا لكم انتن انتم كنت كان كانت كانوا تكون تكونين يكون يكونوا ليس ليست ليسوا
""".split())


In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("ـ", "", text)  # remove tatweel
    return text

def clean_text(text):
    # تحويل للنص لو فيه None
    if type(text) != str:
        text = ""

    # إزالة الروابط و mentions و hashtags
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)

    # إزالة التشكيل
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)

    # إزالة elongation
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # إزالة أرقام
    text = re.sub(r'\d+', '', text)

    # إزالة الرموز وعلامات الترقيم
    text = re.sub(r'[^\w\s]', ' ', text)

    # توحيد الحروف
    text = normalize_arabic(text)

    # إزالة stopwords
    text = " ".join(w for w in text.split() if w not in arabic_stopwords)

    # إزالة مسافات زائدة
    text = re.sub(r"\s+", " ", text).strip()

    return text

# تطبيق التنظيف
df["clean_text"] = df["text"].apply(clean_text)
df.head()


,text,label,clean_text
0,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,1,الهلال_الاهلي فوز هلالي مهم الحمد لله زوران بي...
1,صباحك خيرات ومسرات 🌸,1,صباحك خيرات ومسرات
2,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,1,تامل قال الله ﷻ _ بواد ذي زرع _ ومع هتف بالدعا...
3,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,1,يا جدعان الرجاله اللي ال دول خطر ع تويتر وربنا...
4,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,1,رساله صباحيه اللهم اسالك التوفيق امورنا واكتب ...


In [ ]:
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

model_cv = MultinomialNB()
model_cv.fit(X_train_cv, y_train)

y_pred_cv = model_cv.predict(X_test_cv)
acc_cv = accuracy_score(y_test, y_pred_cv)

print("📌 CountVectorizer Accuracy:", acc_cv)


📌 CountVectorizer Accuracy: 0.7439236111111112


In [ ]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
acc_tfidf = accuracy_score(y_test, y_pred_tfidf)

print("📌 TF-IDF Accuracy:", acc_tfidf)


📌 TF-IDF Accuracy: 0.7443576388888888
